# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [129]:
!head smsspamcollection.csv

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [130]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,-1
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,-1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [131]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [132]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [133]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 138943.68it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [134]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1,  1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

In [135]:
df_train.iloc[1,:]

text     Ok lar... Joking wif u oni...
label                               -1
Name: 1, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [136]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.1%
Coverage for free(): 4.6%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [137]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.0,0.0
free,1,[1],0.046433,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [138]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
2626,"FREE RING TONE just text ""POLYS"" to 87131. Then every week get a new tone. 0870737910216yrs only £1.50/wk.",-1
4543,FreeMsg Hi baby wow just got a new cam moby. Wanna C a hot pic? or Fancy a chat?Im w8in 4uTxt / rply CHAT to 82242 Hlp 08712317606 Msg150p 2rcv,-1
4236,FREEMSG: Our records indicate you may be entitled to 3750 pounds for the Accident you had. To claim for free reply with YES to this msg. To opt out text STOP,-1
3333,Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free,-1
389,4mths half price Orange line rental & latest camera phones 4 FREE. Had your phone 11mths ? Call MobilesDirect free on 08000938767 to update now! or2stoptxt,-1
3819,74355 XMAS iscoming & ur awarded either £500 CD gift vouchers & free entry 2 r £100 weekly draw txt MUSIC to 87066 TnC,-1
3424,Had your mobile 10 mths? Update to latest Orange camera/video phones for FREE. Save £s with Free texts/weekend calls. Text YES for a callback orno to opt out,-1
1777,Call FREEPHONE 0800 542 0578 now!,-1
1663,Hi if ur lookin 4 saucy daytime fun wiv busty married woman Am free all next week Chat now 2 sort time 09099726429 JANINExx Calls£1/minMobsmoreLKPOBOX177HP51FL,-1
659,"It doesnt make sense to take it there unless its free. If you need to know more, wikipedia.com",-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [139]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 119137.08it/s]


In [140]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.0
free,1,[1],0.046433,0.000897,0.0
call_now,2,[1],0.003140,0.000897,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [141]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets

{(-1, -1): array([   0,    1,    3, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   2,    5,    8,   11,   47,   63,   75,  124,  143,  148,  156,
         202,  215,  235,  253,  286,  296,  309,  313,  324,  366,  415,
         468,  481,  488,  505,  532,  545,  576,  602,  635,  638,  647,
         648,  656,  704,  729,  766,  822,  885,  922,  991,  993, 1028,
        1099, 1146, 1159, 1167, 1193, 1205, 1206, 1212, 1228, 1235, 1262,
        1288, 1303, 1313, 1331, 1352, 1414, 1426, 1429, 1433, 1440, 1486,
        1526, 1533, 1542, 1556, 1583, 1589, 1608, 1647, 1673, 1678, 1700,
        1736, 1783, 1835, 1844, 1853, 1879, 1893, 1899, 1901, 1985, 2011,
        2020, 2049, 2066, 2071, 2102, 2118, 2133, 2157, 2244, 2248, 2276,
        2285, 2300, 2303, 2361, 2384, 2400, 2406, 2409, 2496, 2500, 2502,
        2536, 2567, 2607, 2643, 2658, 2683, 2699, 2721, 2730, 2733, 2734,
        2742, 2747, 2793, 2804, 2868, 2873, 2922, 2935, 2948, 2964, 2980,
        3008, 3013, 3022, 3051, 3077, 30

In [142]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
1502,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
2850,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call",-1
2992,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
3893,URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893. ACL03530150PM,-1
4073,Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821,-1
4108,"HOT LIVE FANTASIES call now 08707500020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
4797,URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893,-1
5468,"URGENT! Last weekend's draw shows that you have won £1000 cash or a Spanish holiday! CALL NOW 09050000332 to claim. T&C: RSTM, SW7 3SS. 150ppm",-1
5481,Shall call now dear having food,-1


In [143]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.0
free,1,[1],0.046433,0.000897,0.0
call_now,2,[1],0.003140,0.000897,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [144]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [145]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"I\s.*free", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 70641.91it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.000000,0.000000
free,1,[1],0.046433,0.046433,0.006057
call_now,2,[1],0.003140,0.000897,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.006057


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [146]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

,text,label
3855,oh ya... Got hip hop open. Haha i was thinking can go for jazz then zoom to cine... Actually tonight i'm free leh... And there's a kb lesson tonight,-1
3595,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 08000930705,-1
3951,I got to video tape pple type in message lor. U so free wan 2 help me? Hee... Cos i noe u wan 2 watch infernal affairs so ask u along. Asking shuhui oso.,-1
1364,"Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping. Can you just please find some other way to get what you wanted us to get. Please forgive me. You can reply free via yahoo messenger.",-1
4949,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",-1
3333,Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free,-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
4471,"Lemme know when I can swing by and pick up, I'm free basically any time after 1 all this semester",-1
1663,Hi if ur lookin 4 saucy daytime fun wiv busty married woman Am free all next week Chat now 2 sort time 09099726429 JANINExx Calls£1/minMobsmoreLKPOBOX177HP51FL,-1
948,Hey i booked the kb on sat already... what other lessons are we going for ah? Keep your sat night free we need to meet and confirm our lodging,-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [147]:
@labeling_function()
def contains_money(sms):
    if (re.search(r"[0-9]+(\.[0-9]+)?[\t \s]?[\$£€]", sms.text) or
        re.search(r"[\$£€][\t \s]?[0-9]+(\.[0-9]+)?", sms.text)):
        return SPAM
    else:
        return ABSTAIN

### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [148]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 50721.72it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.001795,0.000000
free,1,[1],0.046433,0.046433,0.006057
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.006057
has_many_uppercase_words,4,[1],0.178107,0.019067,0.001795


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [149]:
@labeling_function()
def short_and_no_uppercase(sms):
    return HAM if len(sms.text) < 10 and not any([word.isupper() for word in sms.text.split()])  else ABSTAIN

### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [150]:
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [151]:
!python3 -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.5.3) =================
ℹ spaCy installation:
/opt/homebrew/lib/python3.11/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.5.0,<3.6.0   3.5.0   ✔
en_core_web_md   >=3.5.0,<3.6.0   3.5.0   ✔



In [152]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [153]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents:
    print(e.text, e.label_)

England GPE
the United Kingdom GPE
Wales ORG
Scotland GPE
The Irish Sea LOC
England GPE
the Celtic Sea LOC
England GPE
Europe LOC
the North Sea LOC
English LANGUAGE
five-eighths CARDINAL
Great Britain GPE
the North Atlantic LOC
over 100 CARDINAL
the Isles of Scilly GPE
the Isle of Wight GPE


In [154]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [155]:
df_train.columns

Index(['text', 'label'], dtype='object')

In [156]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [157]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:19<00:00, 226.39it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.003140,0.001346
free,1,[1],0.046433,0.046433,0.007402
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.007402
has_many_uppercase_words,4,[1],0.178107,0.025123,0.007851
has_person,5,[0],0.051144,0.008748,0.008748


Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [158]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [159]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [160]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 9756.46it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.003589,0.001346
free,1,[1],0.046433,0.046433,0.007402
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.007402
has_many_uppercase_words,4,[1],0.178107,0.045087,0.007851
has_person,5,[0],0.051144,0.016375,0.016375
many_rare_words,6,[1],0.064379,0.035666,0.009421


In [161]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

,text,label
3767,Yes.mum lookin strong:),-1
2692,Hey tmr meet at bugis 930 ?,-1
1579,How to Make a girl Happy? It's not at all difficult to make girls happy. U only need to be... 1. A friend 2. Companion 3. Lover 4. Chef . . . &lt;#&gt; . Good listener &lt;#&gt; . Organizer &lt;#&gt; . Good boyfriend &lt;#&gt; . Very clean &lt;#&gt; . Sympathetic &lt;#&gt; . Athletic &lt;#&gt; . Warm . . . &lt;#&gt; . Courageous &lt;#&gt; . Determined &lt;#&gt; . True &lt;#&gt; . Dependable &lt;#&gt; . Intelligent . . . &lt;#&gt; . Psychologist &lt;#&gt; . Pest exterminator &lt;#&gt; . Psychiatrist &lt;#&gt; . Healer . . &lt;#&gt; . Stylist &lt;#&gt; . Driver . . Aaniye p...,-1
3915,Today is ACCEPT DAY..U Accept me as? Brother Sister Lover Dear1 Best1 Clos1 Lvblefrnd Jstfrnd Cutefrnd Lifpartnr Belovd Swtheart Bstfrnd No rply means enemy,-1
926,"K, wait chikku..il send aftr &lt;#&gt; mins",-1
3925,How? Izzit still raining?,-1
4390,"K I'm ready, &lt;#&gt; ?",-1
1249,"HI HUN! IM NOT COMIN 2NITE-TELL EVERY1 IM SORRY 4 ME, HOPE U AVA GOODTIME!OLI RANG MELNITE IFINK IT MITE B SORTED,BUT IL EXPLAIN EVERYTHIN ON MON.L8RS.x",-1
1542,Do u konw waht is rael FRIENDSHIP Im gving yuo an exmpel: Jsut ese tihs msg.. Evrey splleing of tihs msg is wrnog.. Bt sitll yuo can raed it wihtuot ayn mitsake.. GOODNIGHT &amp; HAVE A NICE SLEEP..SWEET DREAMS..,-1
1291,Eat jap done oso aft ur lect wat... Ü got lect at 12 rite...,-1


#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [162]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

Yetunde    PROPN      NNP        Yetunde   
,          PUNCT      ,          ,         
i          PRON       PRP        I         
'm         AUX        VBP        be        
sorry      ADJ        JJ         sorry     
but        CCONJ      CC         but       
moji       ADJ        JJ         moji      
and        CCONJ      CC         and       
i          PRON       PRP        I         
seem       VERB       VBP        seem      
too        ADV        RB         too       
busy       ADJ        JJ         busy      
to         PART       TO         to        
be         AUX        VB         be        
able       ADJ        JJ         able      
to         PART       TO         to        
go         VERB       VB         go        
shopping   NOUN       NN         shopping  
.          PUNCT      .          .         


In [176]:
@preprocessor(memoize=True)
def adj_cnt(sms):
    sms.adj_cnt = sum(token.pos_ == "ADJ" for token in nlp(sms.text))
    return sms

In [179]:
@labeling_function(pre=[adj_cnt])
def more_than_3_adj(sms):
    return SPAM if sms.adj_cnt > 3 else ABSTAIN

In [180]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words, more_than_3_adj]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:19<00:00, 231.15it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.004038,0.001346
free,1,[1],0.046433,0.046433,0.007402
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.007402
has_many_uppercase_words,4,[1],0.178107,0.047779,0.007851
has_person,5,[0],0.051144,0.016375,0.016375
many_rare_words,6,[1],0.064379,0.036115,0.009421
more_than_3_adj,7,[1],0.035218,0.010543,0.001346


## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [257]:
# lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words, hot, contains_money, short_and_no_uppercase, more_than_3_adj]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 1114/1114 [00:00<00:00, 27021.22it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011440,0.003589,0.001346
free,1,[1],0.046433,0.046433,0.007402
call_now,2,[1],0.003140,0.002692,0.000000
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.007402
has_many_uppercase_words,4,[1],0.178107,0.058771,0.007851
has_person,5,[0],0.051144,0.016375,0.016375
many_rare_words,6,[1],0.064379,0.039480,0.012337
hot,7,[1],0.001122,0.001122,0.000000
contains_money,8,[1],0.048228,0.031404,0.000673
short_and_no_uppercase,9,[0],0.009197,0.002916,0.002916


In [258]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.008079,0.001795,0.000000
free,1,[1],0.052065,0.052065,0.004488
call_now,2,[1],0.007181,0.007181,0.000000
regex_I_am_free,3,"[0, 1]",0.052065,0.052065,0.004488
has_many_uppercase_words,4,[1],0.206463,0.061041,0.008079
has_person,5,[0],0.043088,0.013465,0.013465
many_rare_words,6,[1],0.059246,0.028725,0.008977
hot,7,[1],0.001795,0.001795,0.000000
contains_money,8,[1],0.046679,0.031418,0.000000
short_and_no_uppercase,9,[0],0.008977,0.001795,0.001795


In [259]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [260]:
preds_train

array([-1,  1,  1, ..., -1, -1, -1])

In [261]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 3153
LABEL: 0, count: 183
LABEL: 1, count: 1122


In [262]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.032]
INFO:root:[100 epochs]: TRAIN:[loss=0.004]
INFO:root:[200 epochs]: TRAIN:[loss=0.003]
INFO:root:[300 epochs]: TRAIN:[loss=0.002]
INFO:root:[400 epochs]: TRAIN:[loss=0.001]
100%|██████████| 500/500 [00:00<00:00, 6082.12epoch/s]
INFO:root:Finished Training


In [263]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority voting accuracy: 46.9%
Probabilistic model accuracy: 47.1%


Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [264]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

((4458, 2), (1397, 2))

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [265]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [266]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0)

In [267]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

Logistic regression accuracy: 23.2%


As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [268]:
import spacy

nlp = spacy.load('en_core_web_sm')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

Entities: [('sec', 'ORG')]
Entities: []
Entities: []
Entities: []
Entities: [('south tampa', 'LOC')]
Entities: [('750', 'CARDINAL'), ('150', 'CARDINAL'), ('08002988890', 'CARDINAL')]
Entities: []
Entities: [('juz dun', 'PERSON'), ('2', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('Babe', 'PERSON')]
Entities: [('Call 09095350301', 'ORG'), ('60p', 'ORDINAL')]
Entities: [('4', 'CARDINAL')]
Entities: [('dis name', 'ORG')]
Entities: [('night', 'TIME')]
Entities: []
Entities: [('Sorry de', 'PERSON')]
Entities: []
Entities: [('3', 'CARDINAL')]
Entities: [('83110', 'DATE')]
Entities: [('yahoo', 'ORG')]
Entities: []
Entities: [('Halloween', 'DATE'), ('CARD', 'ORG'), ('8007', 'DATE')]
Entities: []
Entities: []
Entities: [('Shifad', 'PERSON')]
Entities: [('two', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: [('5226', 'DATE'), ('350', 'CARDINAL'), ('08712300220', 'CARDINAL'), ('1131', 'DATE')]
Entities: [('Love', 'WORK_OF_ART')]
Entities

In [279]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

['jurong point',
 'Nah',
 'Mark',
 'Yummy',
 'Mark',
 'Mallika Sherawat',
 'Matrix3',
 'Divorce Barbie',
 'Ken',
 'Wah']

In [280]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacy])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [281]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dominikludwiczak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [282]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [283]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [304]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [305]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

,TF Name,Original Text,Transformed Text
0,random_person_ner,OH FUCK. JUSWOKE UP IN A BED ON A BOATIN THE DOCKS. SLEPT WID 25 YEAR OLD. SPINOUT! GIV U DA GOSSIP L8R. XXX,OH FUCK. JUSWOKE UP IN A BED ON A BOATIN THE DOCKS. SLEPT WID Yar. SPINOUT! GIV U DA GOSSIP L8R. XXX
1,random_person_ner,2marrow only. Wed at &lt;#&gt; to 2 aha.,Ya only. Wed at &lt;#&gt; to 2 aha.
2,random_person_ner,Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GBP/mnth inc 3hrs 16 stop?txtStop www.gamb.tv,Txt: CALL to No: 86888 & claim your reward of Hahaha talk time to use from your phone now! Jay/mnth inc CLAIRE stop?txtStop www.gamb.tv
3,random_person_ner,"I call you later, don't have network. If urgnt, sms me.","I call you later, don't have network. If urgnt, sms me."
4,random_person_ner,I cant pick the phone right now. Pls send a message,I cant pick the phone right now. Pls send a message
...,...,...,...
887,replace_words_with_synonym,S s..first time..dhoni rocks...,S s..first time..dhoni rocks...
888,replace_words_with_synonym,No got new job at bar in airport on satsgettin 4.47per hour but means no lie in! keep in touch,No got new job at bar in airport on satsgettin 4.47per hour but means no lie in! keep in touch
889,replace_words_with_synonym,Its a site to simulate the test. It just gives you very tough questions to test your readiness.,Its a site to simulate the test. It just gives you very tough questions to test your readiness.
890,replace_words_with_synonym,R u over scratching it?,roentgen u over scratching it?


Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [306]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

100%|██████████| 446/446 [00:10<00:00, 41.39it/s]


In [307]:
df_train_sample.shape, df_train_augmented.shape

((446, 2), (1338, 2))

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).

In [308]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        if(nlp(str(words[word_idx]))[0].pos_ == "VERB"):
            synonym = get_synonym(words[word_idx])
            if synonym:
                words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms